In [1]:
import healpy as hp
import matplotlib.pyplot as plt
import sys
import numpy as np
from tqdm import tqdm
sys.path.append('../../')

In [2]:
%load_ext autoreload
%autoreload 2
from cobi.simulation import LATsky
from cobi.spectra import Spectra
from cobi.mle import MLE

In [3]:
libdir = '/global/cfs/cdirs/sobs/cosmic_birefringence/v0'
nside = 2048
cb_model = "iso"
beta = 0.35
alpha = [-0.1,-0.1,0.2,0.2,.15,.15]
alpha_err = 0.1

In [4]:
lat = LATsky(libdir, nside, cb_model, beta, alpha=alpha,alpha_err=alpha_err, bandpass=True,verbose=True)

CMB : INFO - Loading CMB power spectra from file
CMB : INFO - Isotropic(constant) cosmic birefringence model selected
Mask : INFO - Upgrading mask LAT resolution
Noise : INFO - Noise Model:[LAT] White + 1/f noise v3.1.1


In [5]:
# for i in range(100):
#     if lat.checkObsQU(i,what='filesize'):
#         pass
#     else:
#         print(f"Simulating {i}")

In [6]:
spec = Spectra(lat, libdir, cache=True,parallel=1)
#spec.Compute(3 , sync=True)

Spectra : INFO - Reading apodised mask from /global/cfs/cdirs/sobs/cosmic_birefringence/v0/spectra_2048_aposcale2p0/workspaces/mask_N2048_aposcale2p0_CO_PS.fits
Spectra : INFO - Reading coupling Matrix from /global/cfs/cdirs/sobs/cosmic_birefringence/v0/spectra_2048_aposcale2p0/workspaces/coupling_matrix_N2048_fsky0p55_aposcale2p0_bw1_CO_PS.fits
Spectra : INFO - Parallelized single loop


In [22]:
h = []
for i in range(100):
    if all(spec.sync_x_obs_check(i)):
        continue
    h.append(i)

In [23]:
len(h)

0

In [24]:
fit = "Ad + beta + alpha"
binwidth = 10
bmin = 50
bmax = 2000
mle = MLE(libdir,spec,fit, alpha_per_split=False,rm_same_tube=True,binwidth=binwidth,bmin=bmin,bmax=bmax)

CMB : INFO - Loading CMB power spectra from file
CMB : INFO - Isotropic(constant) cosmic birefringence model selected


Fitting a common polarisation angle per frequency
Don't use cross-spectra of bands within the same optical tube


In [25]:
beta = []
for i in range(100):
    beta.append(mle.estimate_angles(i)['beta'])
beta = np.array(beta)

In [26]:
np.mean(beta)

0.36736355156712414

In [27]:
np.std(beta)

0.07572409590188695